In [99]:
from skimage import io, feature, filters, transform
import os
import matplotlib.pyplot as plt
import numpy as np
import sep
from scipy import ndimage
from skimage.registration import phase_cross_correlation
from scipy.spatial import KDTree, distance

In [100]:
%matplotlib qt

In [101]:
def get_cluster_coords(image):
    threshold = filters.threshold_otsu(image)
    binary = image > threshold
    label_image, num_labels = ndimage.label(binary)
    center_of_masses = ndimage.center_of_mass(binary, label_image, range(num_labels+1))
    return center_of_masses[1:]

In [102]:
def do_constellation_mapping(se_cluster_pos, fq_cluster_pos):
    ##performing the classify hits algorithm, starting with making KDTree of se and aligned fq clusters
    se_cluster_tree = KDTree(se_cluster_pos)
    fq_cluster_tree = KDTree(fq_cluster_pos)
    
    #creating mapping sets
    se_to_fq= set()
    fq_to_se= set()
    
    for i, pt in enumerate(se_cluster_pos):
        dist, idx = fq_cluster_tree.query(pt)
        se_to_fq.add((i, idx))
        
    for i, pt in enumerate(fq_cluster_pos):
        dist, idx = se_cluster_tree.query(pt)
        fq_to_se.add((idx, i))
        
    mutual_hits = se_to_fq & fq_to_se
    non_mutual_hits = se_to_fq ^ fq_to_se
    
    return (mutual_hits, non_mutual_hits)

In [103]:
in_folder = 'assets'
in_file = 'big_dipper_blue_ch.tif'

source_image = io.imread(os.path.join(in_folder, in_file))
plt.imshow(source_image, cmap='gray')

In [104]:
source_image.shape

(1623, 1293)

In [105]:
blobs_log = feature.blob_log(source_image, min_sigma=20, max_sigma=30, num_sigma=10, threshold=.1)
blobs_log[:, 2] = blobs_log[:, 2] * np.sqrt(2)
blobs_log.shape

(10, 3)

In [106]:
fig, ax = plt.subplots()
ax.imshow(source_image, cmap='gray')
for blob in blobs_log:
    y, x, r = blob
    c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
    ax.add_patch(c)

In [107]:
point_source_im = np.zeros_like(source_image, dtype=np.float32)
point_source_im[blobs_log.astype(np.int16)[:, 0], blobs_log.astype(np.int16)[:, 1]] = 1
gauss_source_im = ndimage.gaussian_filter(point_source_im, sigma=5)
plt.imshow(gauss_source_im, cmap='gray')

In [108]:
blobs_log

array([[ 569.        ,  665.        ,   28.28427125],
       [ 202.        ,  428.        ,   28.28427125],
       [ 391.        ,  635.        ,   28.28427125],
       [1158.        ,  792.        ,   28.28427125],
       [1072.        ,  982.        ,   28.28427125],
       [ 918.        ,  610.        ,   28.28427125],
       [ 779.        ,  719.        ,   28.28427125],
       [1285.        ,  348.        ,   28.28427125],
       [ 647.        , 1220.        ,   28.28427125],
       [ 254.        , 1058.        ,   28.28427125]])

In [109]:
ref_im_file_name = 'big_dipper_ref.tif'
ref_im = io.imread(os.path.join(in_folder, ref_im_file_name))
plt.imshow(ref_im, cmap='gray')

In [114]:
radius = 1500 #radius for the polar warp 
source_im_polar = transform.warp_polar(gauss_source_im, radius=radius, scaling='log')
ref_im_polar = transform.warp_polar(ref_im, radius=radius, scaling='log')


shifts, error, phasediff = phase_cross_correlation(source_im_polar, ref_im_polar, upsample_factor=20)
shiftr, shiftc = shifts[:2]

# Calculate scale factor from translation
klog = radius / np.log(radius)
shift_scale = 1 / (np.exp(shiftc / klog))

print(f'rotation: {shiftr}')
print(f'scaling difference: {shift_scale}')

rotation: 53.7
scaling difference: 2.200333482301876


In [115]:
scaled_im = transform.rescale(transform.rotate(gauss_source_im, shiftr), shift_scale)
center_of_masses = get_cluster_coords(scaled_im)
src = get_cluster_coords(ref_im)
dst = center_of_masses
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(ref_im, cmap='gray')
for center in src:
    c = plt.Circle((center[1], center[0]), 25, color='red', linewidth=2, fill=False)
    ax[0].add_patch(c)
ax[1].imshow(scaled_im, cmap='gray')
for center in dst:
    c = plt.Circle((center[1], center[0]), 25, color='red', linewidth=2, fill=False)
    ax[1].add_patch(c)

In [116]:
mutual_hits, non_mutual_hits = do_constellation_mapping(src, dst)
for hit in mutual_hits:
    print(src[hit[0]], dst[hit[1]])

(1436.0958762886598, 1017.6876288659794) (1436.0958762886598, 1017.6876288659794)
(1988.1938144329897, 1564.8762886597938) (1988.1938144329897, 1564.8762886597938)
(1613.8876288659794, 1460.2824742268042) (1613.8876288659794, 1460.2824742268042)
(1978.0887512899897, 2227.514963880289) (1978.0887512899897, 2227.514963880289)
(1257.3264675592172, 662.9155509783728) (1257.3264675592172, 662.9155509783728)
(1378.2466460268317, 58.076367389060884) (1378.2466460268317, 58.076367389060884)
(1529.129363449692, 2322.5) (1529.129363449692, 2322.5)


In [126]:
def right_rotation_matrix(angle, degrees=True):
    if degrees:
        angle *= np.pi / 180.0
    sina = np.sin(angle)
    cosa = np.cos(angle)
    return np.array([[cosa, sina],
                     [-sina, cosa]])

scaled_im = np.zeros(shape=(np.array(gauss_source_im.shape) * shift_scale).astype(np.int16))
scaled_coords = blobs_log[:, :2]*shift_scale
scaled_rotated_coords = np.dot(scaled_coords, right_rotation_matrix(53.7, degrees=True))
r_min = np.min(scaled_rotated_coords[:, 0])
c_min = np.min(scaled_rotated_coords[:, 1])
print(r_min, c_min)
scaled_rotated_coords = scaled_rotated_coords - np.tile(np.array([r_min, c_min]), reps=(scaled_rotated_coords.shape[0], 1))
print(scaled_rotated_coords.astype(np.int16))

-1545.2959103277656 915.732926170701
[[1107  959]
 [1049    0]
 [ 928  604]
 [1649 2169]
 [1200 2264]
 [1659 1506]
 [1285 1402]
 [2602 1816]
 [ 224 1820]
 [   0  912]]


In [124]:
scaled_im[scaled_rotated_coords.astype(np.int16)[:, 0], scaled_rotated_coords.astype(np.int16)[:, 1]] = 1
gauss_scaled_im = ndimage.gaussian_filter(scaled_im, sigma=5)
fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True)
ax[0].imshow(ref_im, cmap='gray')
for center in src:
    c = plt.Circle((center[1], center[0]), 25, color='red', linewidth=2, fill=False)
    ax[0].add_patch(c)
ax[1].imshow(gauss_scaled_im, cmap='gray')
for center in scaled_rotated_coords.astype(np.int16):
    c = plt.Circle((center[1], center[0]), 25, color='red', linewidth=2, fill=False)
    ax[1].add_patch(c)

In [97]:
src = get_cluster_coords(ref_im)
dst = scaled_rotated_coords
mutual_hits, non_mutual_hits = do_constellation_mapping(src, dst)
for hit in mutual_hits:
    print(src[hit[0]], dst[hit[1]])

(2548.4817898022893, 1743.2330905306972) [2602.05863675 1816.28568188]
(445.03108808290153, 942.2466321243523) [  0.        912.8668153]
(1714.5960334029228, 1582.5960334029228) [1659.38726313 1506.76867268]
(1252.570385818561, 1463.8133472367049) [1285.03129362 1402.26472564]


/opt/anaconda3/envs/regular/lib/python3.8/site-packages/scipy/ndimage/measurements.py:1406: RuntimeWarning: invalid value encountered in true_divide
  results = [sum(input * grids[dir].astype(float), labels, index) / normalizer
